In [391]:
import numpy as np 
import pandas as pd 
import emoji
import os
import keras
import preprocessor as p
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM,Bidirectional, Embedding
from keras.layers.core import Dense, Activation, Dropout
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import DistilBertModel, DistilBertTokenizer
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [392]:
data = pd.read_csv('text_emotion.csv')

In [393]:
data=data[['sentiment','content']]
data

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
39995,neutral,@JohnLloydTaylor
39996,love,Happy Mothers Day All my love
39997,love,Happy Mother's Day to all the mommies out ther...
39998,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [394]:
misspell_data = pd.read_csv("aspell.txt",sep=":",names=["correction","misspell"])
misspell_dict=dict()
for misspell in misspell_data['misspell']:
    key=misspell.split()[0]
    misspell_dict[key] = misspell_data.loc[misspell_data['misspell']==misspell,'correction'].values[0]

In [395]:
def misspelled_correction(val):
    for x in val.split(): 
        if x in misspell_dict.keys(): 
            val = val.replace(x, misspell_dict[x]) 
    return val

data["clean_content"] = data['content'].apply(lambda x : misspelled_correction(x))
data = data.drop('content', axis=1)

In [396]:
contractions = pd.read_csv("contractions.csv")
cont_dict = dict(zip(contractions.Contraction, contractions.Meaning))

In [397]:
def contraction(val): 
    for x in val.split(): 
        if x in cont_dict.keys(): 
            val = val.replace(x, cont_dict[x]) 
    return val

In [398]:
data['clean_content'] = data['clean_content'].apply(lambda x : contraction(x))

In [399]:
#URL, and @ remover from preprocessor library
p.set_options(p.OPT.MENTION, p.OPT.URL)
p.clean("hello guys https://github.com/s/preprocessor")

'hello guys'

In [400]:
data['clean_content']=data['clean_content'].apply(lambda x : p.clean(x))

In [401]:
def punctuation(val): 
    punctuations = '''()-[]{};:'"\,<>./@#$%^&_~'''
    for x in val.lower(): 
        if x in punctuations: 
            val = val.replace(x, "") 
    return val

In [402]:
punctuation("me ''()-[]{};:'\,<>./@#$%^&_~' ? !  ")

'me  ? !  '

In [403]:
#punctuation and emojis
data['clean_content'] = data['clean_content'].apply(lambda x : ' '.join(punctuation(emoji.demojize(x)).split()))

In [404]:
#lowercase
data['clean_content']=data['clean_content'].str.lower()
#empty comments
data = data[data['clean_content'] != ""]
data

,sentiment,clean_content
0,empty,i know i was listenin to bad habit earlier and...
1,sadness,layin n bed with a headache ughhhhwaitin on yo...
2,sadness,funeral ceremonygloomy friday
3,enthusiasm,wants to hang out with friends soon!
4,neutral,we want to trade with someone who has houston ...
...,...,...
39994,happiness,succesfully following tayla!!
39996,love,happy mothers day all my love
39997,love,happy mothers day to all the mommies out there...
39998,happiness,wassup beautiful!!! follow me!! peep out my ne...


In [405]:
# # Download the stopwords corpus if not already downloaded
# nltk.download('stopwords')
# nltk.download('punkt')
# # Define the set of stopwords
# stop_words = set(stopwords.words('english'))

# # Function to remove stopwords from a text string
# def remove_stopwords(text):
#     tokens = word_tokenize(text)  # Tokenize the text into words
#     filtered_tokens = [word for word in tokens if word.lower() not in stop_words]  # Filter out stopwords
#     filtered_text = ' '.join(filtered_tokens)  # Reconstruct the text without stopwords
#     return filtered_text

# # Apply the remove_stopwords function to the 'Text' column
# data['clean_content'] = data['clean_content'].apply(remove_stopwords)


In [406]:
data

,sentiment,clean_content
0,empty,i know i was listenin to bad habit earlier and...
1,sadness,layin n bed with a headache ughhhhwaitin on yo...
2,sadness,funeral ceremonygloomy friday
3,enthusiasm,wants to hang out with friends soon!
4,neutral,we want to trade with someone who has houston ...
...,...,...
39994,happiness,succesfully following tayla!!
39996,love,happy mothers day all my love
39997,love,happy mothers day to all the mommies out there...
39998,happiness,wassup beautiful!!! follow me!! peep out my ne...


In [407]:
data['sentiment'].value_counts()

neutral       8579
worry         8454
happiness     5208
sadness       5162
love          3841
surprise      2187
fun           1776
relief        1526
hate          1323
empty          815
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [408]:
labels= {"neutral":0, "worry":1,"happiness":2,"sadness":3,"love":4,"surprise":5,"fun":6,"relief":7,"hate":8,"empty":9,"enthusiasm":10,"boredom":11,"anger":12}

In [409]:
data["sentiment"] = data['sentiment'].replace(labels)


In [410]:
data

,sentiment,clean_content
0,9,i know i was listenin to bad habit earlier and...
1,3,layin n bed with a headache ughhhhwaitin on yo...
2,3,funeral ceremonygloomy friday
3,10,wants to hang out with friends soon!
4,0,we want to trade with someone who has houston ...
...,...,...
39994,2,succesfully following tayla!!
39996,4,happy mothers day all my love
39997,4,happy mothers day to all the mommies out there...
39998,2,wassup beautiful!!! follow me!! peep out my ne...


In [411]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data['sentiment'])
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Y = onehot_encoder.fit_transform(integer_encoded)

C:\Users\Chanakya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [412]:
X_train, X_testval, y_train, y_testval = train_test_split(data['clean_content'],Y, random_state=42, test_size=0.2, shuffle=True)
X_test, X_val,y_test, y_val = train_test_split(X_testval,y_testval, random_state=42, test_size=0.5, shuffle=True)
print(len(X_train),len(X_test),len(X_val))

31935 3992 3992


In [413]:
maxl=0
for content in data['clean_content']:
    maxl=max(maxl,len(content.split()))
print(maxl)

34


In [414]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
token.fit_on_texts(list(X_train) + list(X_test)+list(X_val))
X_train_pad = pad_sequences(token.texts_to_sequences(X_train), maxlen=maxl+1)
X_val_pad = pad_sequences(token.texts_to_sequences(X_val), maxlen = maxl+1)
X_test_pad = pad_sequences(token.texts_to_sequences(X_test), maxlen=maxl+1)

In [415]:
word2idx = token.word_index
print(len(word2idx))

35574


## Bidirectional LSTM

In [416]:
embeddings_index = {}
f = open(os.path.join('glove.6B.200d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs= np.asarray(values [1:], dtype='float32')
    embeddings_index [word] = np.array(coefs)
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [417]:
embedding_matrix = np.random.random((len(word2idx) + 1, 200))
for word, i in word2idx.items():
    if word in embeddings_index.keys():
        embedding_matrix[i] = embeddings_index[word]

In [418]:
embed_dim = 200
lstm_out = 256

model_lstm = Sequential()
model_lstm.add(Embedding(len(word2idx) +1 , embed_dim,input_length = X_test_pad.shape[1],weights=[embedding_matrix],trainable=False))
model_lstm.add(SpatialDropout1D(0.2))
model_lstm.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2,return_sequences=True),input_shape=(35,200 )))
model_lstm.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)))
model_lstm.add(keras.layers.core.Dense(13, activation='softmax'))
#adam rmsprop 
model_lstm.compile(loss = "categorical_crossentropy", optimizer='adam',metrics = ['accuracy'])
print(model_lstm.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 35, 200)           7115000   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 35, 200)          0         
 lDropout1D)                                                     
                                                                 
 bidirectional_2 (Bidirectio  (None, 35, 512)          935936    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 512)              1574912   
 nal)                                                            
                                                                 
 dense_24 (Dense)            (None, 13)                6669      
                                                      

In [419]:
batch_size = 32
Epochs=10

In [268]:

model_lstm.fit(X_train_pad, y_train, epochs = Epochs, batch_size=batch_size,validation_data=(X_val_pad, y_val))

Epoch 1/10
998/998 [==============================] - 538s 539ms/step - loss: 1.9370 - accuracy: 0.3358 - val_loss: 1.9341 - val_accuracy: 0.3329
Epoch 2/10
998/998 [==============================] - 529s 530ms/step - loss: 1.8751 - accuracy: 0.3561 - val_loss: 1.9182 - val_accuracy: 0.3392
Epoch 3/10
998/998 [==============================] - 521s 523ms/step - loss: 1.8364 - accuracy: 0.3702 - val_loss: 1.9193 - val_accuracy: 0.3457
Epoch 4/10
998/998 [==============================] - 528s 529ms/step - loss: 1.7921 - accuracy: 0.3809 - val_loss: 1.9068 - val_accuracy: 0.3437
Epoch 5/10
998/998 [==============================] - 530s 531ms/step - loss: 1.7420 - accuracy: 0.3973 - val_loss: 1.9090 - val_accuracy: 0.3487
Epoch 6/10
998/998 [==============================] - 538s 539ms/step - loss: 1.6849 - accuracy: 0.4175 - val_loss: 1.9441 - val_accuracy: 0.3404
Epoch 7/10
998/998 [==============================] - 531s 532ms/step - loss: 1.6136 - accuracy: 0.4390 - val_loss: 1.9762 -

In [420]:
# model_lstm.save('bilstm.h5')
from keras.models import load_model

# Load the saved model
model_lstm = load_model('bilstm.h5')

In [421]:
def clean_text(val):
    val = misspelled_correction(val)
    val = contraction(val)
    val = p.clean(val)
    val = val.lower()
    val = ' '.join(punctuation(emoji.demojize(val)).split())
    return val

In [422]:
def get_sentiment(model,text):
    text = clean_text(text)
    #tokenize
    twt = token.texts_to_sequences([text])
    twt = pad_sequences(twt, maxlen=maxl+1, dtype='int32')
    sentiment = model.predict(twt,batch_size=1,verbose = 2)
    sent = np.round(np.dot(sentiment,100).tolist(),0)[0]
    result = pd.DataFrame([labels.keys(),sent]).T
    result.columns = ["sentiment","percentage"]
    result=result[result.percentage !=0]
    return result

In [424]:
get_sentiment(model_lstm,"I am fed up of you")

1/1 - 0s - 102ms/epoch - 102ms/step


,sentiment,percentage
0,neutral,63.0
1,worry,9.0
2,happiness,8.0
3,sadness,1.0
4,love,1.0
5,surprise,3.0
6,fun,7.0
7,relief,1.0
8,hate,1.0
9,empty,1.0


In [467]:
maxl=36

## DistillBert

In [538]:
AUTO = tf.data.experimental.AUTOTUNE
MODEL='distilbert-base-uncased'
dberttokenizer = AutoTokenizer.from_pretrained(MODEL)

In [539]:
def dbertencoder(texts,maxlen):
    dbencoder = dberttokenizer.batch_encode_plus( texts, 
    return_attention_mask=False, 
    return_token_type_ids=False,
    pad_to_max_length=True,
    max_length=maxlen
    )
    return np.array(dbencoder['input_ids'])

X_train_pdbert =pad_sequences(dberttokenizer(X_train.tolist(),padding=False)['input_ids'], maxlen=maxl,padding='post')
X_val_pdbert =pad_sequences(dberttokenizer(X_val.tolist(),padding=False)['input_ids'], maxlen=maxl,padding='post')
X_test_pdbert =pad_sequences(dberttokenizer(X_test.tolist(),padding=False)['input_ids'], maxlen=maxl,padding='post')

In [540]:
#Sample of the distilbert tokenizer
print((X_val_pdbert[49]))
print(X_train.tolist()[1])
print(dberttokenizer.decode(X_val_pdbert[49], skip_special_tokens=False))

[  101  2038  2012  2560  2382 11829 15424  2006  2014  3456   102     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]
yaa! hi amy im showing my new friends our youtube
[CLS] has at least 30 bug bites on her legs [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [541]:
#Creating dataset for training testing and validation
train_dataset = (tf.data.Dataset.from_tensor_slices((X_train_pdbert, y_train)).repeat().shuffle(42).batch(batch_size).prefetch(AUTO)
)

val_dataset = (tf.data.Dataset.from_tensor_slices((X_val_pdbert, y_val)).batch(batch_size).cache().prefetch(AUTO))

test_dataset = (tf.data.Dataset.from_tensor_slices((X_test_pdbert, y_test)).batch(batch_size).cache().prefetch(AUTO)
)


In [542]:
distilbert = TFAutoModel.from_pretrained(MODEL)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [543]:
input_word_ids = Input(shape=(36,), dtype=tf.int32, name="input_word_ids")
sequence_output = distilbert(input_word_ids)[0]
cls = sequence_output[:, 0, :]
out = Dense(13, activation='softmax')(cls)
model_dbert = Model(inputs=input_word_ids, outputs=out)
model_dbert.compile(Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model_dbert.summary()

Model: "model_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 36)]             0         
                                                                 
 tf_distil_bert_model_22 (TF  TFBaseModelOutput(last_h  66362880 
 DistilBertModel)            idden_state=(None, 36, 7            
                             68),                                
                              hidden_states=None, att            
                             entions=None)                       
                                                                 
 tf.__operators__.getitem_37  (None, 768)              0         
  (SlicingOpLambda)                                              
                                                                 
 dense_40 (Dense)            (None, 13)                9997      
                                                          

In [544]:
steps_per_epoch = X_train.shape[0]//batch_size
model_dbert.fit(train_dataset,steps_per_epoch=steps_per_epoch,validation_data=val_dataset,epochs=3)

Epoch 1/3
997/997 [==============================] - 1346s 1s/step - loss: 1.8867 - accuracy: 0.3535 - val_loss: 1.8455 - val_accuracy: 0.3625
Epoch 2/3
997/997 [==============================] - 1345s 1s/step - loss: 1.7345 - accuracy: 0.4033 - val_loss: 1.8356 - val_accuracy: 0.3692
Epoch 3/3
997/997 [==============================] - 1334s 1s/step - loss: 1.6449 - accuracy: 0.4373 - val_loss: 1.8681 - val_accuracy: 0.3662


In [512]:
# model_dbert.save('dbert.h5')
model_dbert = load_model('dbert.h5')

ValueError: Unknown layer: TFDistilBertModel. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [545]:
model_dbert.fit(train_dataset,steps_per_epoch=steps_per_epoch,validation_data=val_dataset,epochs=2)

Epoch 1/2
997/997 [==============================] - 1340s 1s/step - loss: 1.5308 - accuracy: 0.4790 - val_loss: 1.9513 - val_accuracy: 0.3562
Epoch 2/2
997/997 [==============================] - 1331s 1s/step - loss: 1.3824 - accuracy: 0.5312 - val_loss: 2.0754 - val_accuracy: 0.3472


In [546]:
def get_sentimentbert(model,text):
    text = clean_text(text)
    #tokenize
    toktext =pad_sequences(dberttokenizer([text],padding=False)['input_ids'], maxlen=maxl,padding='post')
    test1 = (tf.data.Dataset.from_tensor_slices(toktext).batch(1))
    #test1
    sentiment = model.predict(test1,verbose = 0)
    sent = np.round(np.dot(sentiment,100).tolist(),0)[0]
    result = pd.DataFrame([labels.keys(),sent]).T
    result.columns = ["sentiment","percentage"]
    result=result[result.percentage !=0]
    return result

In [549]:
get_sentimentbert(model_dbert,"this is very beautiful")

,sentiment,percentage
0,neutral,10.0
1,worry,2.0
2,happiness,55.0
4,love,21.0
5,surprise,8.0
6,fun,3.0
10,enthusiasm,1.0
